In [1]:
!pip install py-entitymatching

     |████████████████████████████████| 2.0MB 2.1MB/s 
     |████████████████████████████████| 788kB 31.8MB/s 
     |████████████████████████████████| 1.1MB 25.2MB/s 
     |████████████████████████████████| 368kB 29.7MB/s 
     |████████████████████████████████| 665kB 33.6MB/s 
  Created wheel for py-entitymatching: filename=py_entitymatching-0.4.0-cp37-cp37m-linux_x86_64.whl size=2634147 sha256=759951272f6f561fcc67b035816fb4d07a85813d492c09936c92f83e8d1259f2
  Stored in directory: /root/.cache/pip/wheels/88/65/50/a8bc0f26e8a82c931658153c4f37cbf5debc22cb0f61dbcd05
  Created wheel for py-stringsimjoin: filename=py_stringsimjoin-0.3.2-cp37-cp37m-linux_x86_64.whl size=3723626 sha256=bed002bf720eaa3ad4e7400fdd8458cf3df7e32331d12b402e0e7e3f97c5233b
  Stored in directory: /root/.cache/pip/wheels/58/30/8e/6fa463100ff6e8595b131067ec43a118c607f621ac297e7dad
  Created wheel for py-stringmatching: filename=py_stringmatching-0.4.2-cp37-cp37m-linux_x86_64.whl size=2055229 sha256=22b0c98ab9405236146

In [1]:
# https://hpi.de/naumann/projects/repeatability/datasets/restaurants-dataset.html
# http://www.iesl.cs.umass.edu/datasets.html - CORA dataset
# Importing the libraries
import sys
import pandas as pd
import os
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
path_variable = "/content/gdrive/My Drive/BTP/Code/Dataset/Restaurant/"
method = "/content/gdrive/My Drive/BTP/Code/Method1/weights_rest/"

In [3]:
import py_entitymatching as em
# Display the versions
print('python version: ' + sys.version )
print('pandas version: ' + pd.__version__ )
print('magellan version: ' + em.__version__ )

python version: 3.7.10 (default, May  3 2021, 02:48:31) 
[GCC 7.5.0]
pandas version: 1.1.5
magellan version: 0.4.0


In [4]:
# Load csv files as dataframes and set the key attribute in the dataframe
path_A = path_variable+"fodors.csv"
path_B = path_variable+"zagats.csv"
A = em.read_csv_metadata(path_A, key='id')
B = em.read_csv_metadata(path_B, key='id')

# Display number of tuples in the datasets
print('Number of tuples in A: ' + str(len(A)))
print('Number of tuples in B: ' + str(len(B)))
print('Number of tuples in A X B (i.e the cartesian product): ' + str(len(A)*len(B)))

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


Number of tuples in A: 533
Number of tuples in B: 331
Number of tuples in A X B (i.e the cartesian product): 176423


In [5]:
# verify values in S

d1 = dict()
d2 = dict()

t1 = A.to_numpy()
t2 = B.to_numpy()

for x in t1:
  if(x[-1] in d1):
    d1[x[-1]]+=1
  else:
    d1[x[-1]]=1

for x in t2:
  if(x[-1] in d2):
    d2[x[-1]]+=1
  else:
    d2[x[-1]]=1

print("d1 : ",d1)
print("d2 : ",d2)

d1 :  {'American': 165, 'French/Italian/European': 164, 'Asian': 88, 'Diners/Cafes': 35, 'Fusion food(Mexican/mediterranean/caribbean)': 51, 'bbq/fast food': 30}
d2 :  {'American': 70, 'Asian': 69, 'Fusion food(Mexican/mediterranean/caribbean)': 31, 'bbq/fast food': 47, 'French/Italian/European': 70, 'Diners/Cafes': 44}


In [6]:
import pickle
import numpy as np
import torch

dbfile = open(path_variable+'files/feature_matrix_restaurant', 'rb')    
cand_file = open(path_variable+'files/C_restaurant', 'rb')  
S = pickle.load(dbfile)
cand = pickle.load(cand_file)

S = torch.tensor(S.values,dtype=torch.double)
S = S[:,3:]
# S = S.to_numpy(dtype=np.longdouble)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Initial weights would be 1/(num_attributes*num_sim_scores) due to add-to-1 constraint
w_init  = (1 / S.shape[1] )
# W = np.full((S.shape[0], S.shape[1]), w_init,dtype=np.longdouble)
W = torch.DoubleTensor(S.shape[1],1).fill_(w_init)

In [7]:
# ith index of genre_groups denotes genre of ith record pair
genre_groups = []
for i in cand['ltable_group']:
    genre_groups.append(i)

In [20]:
######### Skew metric adopted from https://arxiv.org/pdf/1905.01989.pdf (refer this paper for the metric)

# ith row of RLS matrix will return RLS value of the ith record pair 
# return nx1 matrix, where n=number of record pairs 
def get_RLS(W,S):
    rls = torch.matmul(S,torch.square(W))
    return rls
    # return torch.diag(rls)

def fair_calc(weights,S,flag,k):
  
  rls_matrix = get_RLS(weights,S)
  cand['score'] = rls_matrix.numpy()
  candy = cand.sort_values(by=['score'], ascending=False)

  # print(candy)
  # k=500

  d3 = dict()
  d4 = dict()

  temp = candy.to_numpy()
  cnt = 0

  for _id,pair in enumerate(temp):
    # print(_id)
    # print(pair,pair[11],pair[20])
    if(_id>k):
      # print(_id)
      break

    if(pair[9] in d3):
      d3[pair[9]]+=1
    else:
      d3[pair[9]]=1
      
    if(pair[-2] in d4):
      d4[pair[-2]]+=1
    else:
      d4[pair[-2]]=1
    
    if(pair[9]==(pair[-2])):
      cnt+=1

  
  # print("d3 : ", d3)
  # print("d4 : ",d4)

  # print(cnt)
  
  mini = -1
  maxi = 1
  d_skew = {}
  flagy = 0
  for d in d1.keys():
    if(d in d3.keys()):
      d_skew[d] = np.log( d3[d]*len(t1)/(d1[d]*k))
    
      if(flagy==0):
        flagy+=1
        mini = d_skew[d]
        maxi = d_skew[d]
      
      mini = min(mini,d_skew[d])
      maxi = max(maxi,d_skew[d])
    # else:
    #   print(d)
  
  if(flag):
    print("Group distribution in result : ",d3)
    print("Skew metrix : ", maxi-mini)

  return maxi-mini


# def get_RLS_sum(W,S):
#     rls = torch.matmul(S,torch.transpose(W, 0, 1))
#     return torch.sum(torch.diag(rls))

# this will return a 1D vector of size n , having value which is to be multiplied to 2*w*S 
# returns an nx1 vector
# value at ith index would be multiplied to ith pair's derivative


# def derivative_loss_function(S,W,j,precomputed_value):
#     partial_deriv = 0
#     for i in range(S.shape[0]):
#         num = 2 * S[j,:].dot(W[j,:]) * ( torch.exp(phi * S[i,:].dot(W[i,:])) - torch.exp(phi * (tau - S[i,:].dot(W[i,:]) )) )
#         denom = torch.exp(phi * S[i,:].dot(W[i,:])) + torch.exp(phi * (tau - S[i,:].dot(W[i,:])))
#         partial_deriv += num/denom
#     return partial_deriv


def get_precomputed_values(W,S,phi,tau):
    numerator = torch.exp(phi*get_RLS(W,S)) - torch.exp(phi* (tau-get_RLS(W,S)))
    # print(numerator)
    denominator = torch.exp(phi*get_RLS(W,S)) + torch.exp(phi* (tau-get_RLS(W,S)))
    return torch.divide(numerator,denominator)


def derivative_loss_function(W,S,j,precomputed_values):
    # print(S[:,j].reshape(-1,1).shape)
    # print(torch.transpose(S[:,j],0,1).shape)
    # print(precomputed_values.shape)
    m = torch.matmul(torch.transpose(S[:,j].reshape(-1,1),0,1),precomputed_values)
    der = 2 * W[j,0] * m
    return der

def algorithm_gradient_descent(W,S,learning_rate,iterations,phi,tau,flag_tweak,k):

    for i in range(iterations):
        W_new = torch.zeros(W.shape) 
        sum_W_new = 0
        precomputed_values_em = get_precomputed_values(W,S,phi,tau)
        for j in range(W.shape[0]):
            loss_value = derivative_loss_function(W,S,j,precomputed_values_em)
            W_new[j,0] = W[j,0] - learning_rate * loss_value
            if( W_new[j,0] != float("nan")):
              sum_W_new += W_new[j,0]
        # print(sum_W_new)

        for j in range(W.shape[0]):
            W_new[j,0] = W_new[j,0] / float(sum_W_new)
        W = W_new.double()
        # print(W)
        if((i+1)%10==0):
          print("Iteration " + str(i+1) + " completed ")        
        
        if(flag_tweak):
          f_val = fair_calc(W,S,False,k)
          epsi = 0.1

          for j in range(W.shape[0]):
            W[j,0] += epsi
            f1 = fair_calc(W,S,False,k)
            
            W[j,0] -= 2*epsi
            f2 = fair_calc(W,S,False,k)
            # print(f1,f2,f_val)
            if(f_val<=f1 and f_val<=f2):
              # print("val")
              W[j,0] += epsi

            elif(f1<=f2 and f1<=f_val):
              # print("f1")
              W[j,0] += 2*epsi
        # print(W)

    return W

In [ ]:
#No learning
print(W)
fair_calc(W,S,True,500)

tensor([[0.0500],
        [0.0500],
        [0.0500],
        [0.0500],
        [0.0500],
        [0.0500],
        [0.0500],
        [0.0500],
        [0.0500],
        [0.0500],
        [0.0500],
        [0.0500],
        [0.0500],
        [0.0500],
        [0.0500],
        [0.0500],
        [0.0500],
        [0.0500],
        [0.0500],
        [0.0500]], dtype=torch.float64)
Group distribution in result :  {'American': 98, 'French/Italian/European': 163, 'bbq/fast food': 45, 'Fusion food(Mexican/mediterranean/caribbean)': 58, 'Asian': 78, 'Diners/Cafes': 59}
Skew metrix :  1.0431673776463146


1.0431673776463146

In [21]:
#No tweak of weights
weights1 = algorithm_gradient_descent(W,S,1e-5,100,-100,1,False,500)

Iteration 10 completed 
Iteration 20 completed 
Iteration 30 completed 
Iteration 40 completed 
Iteration 50 completed 
Iteration 60 completed 
Iteration 70 completed 
Iteration 80 completed 
Iteration 90 completed 
Iteration 100 completed 


In [22]:
print(weights1)
fair_calc(weights1,S,True,500)

tensor([[1.8963e-01],
        [2.6903e-01],
        [1.6531e-03],
        [4.2786e-02],
        [1.0905e-02],
        [3.5550e-03],
        [9.6568e-05],
        [8.1715e-04],
        [2.5778e-03],
        [6.1307e-03],
        [7.0380e-05],
        [7.7878e-04],
        [1.6857e-01],
        [2.5376e-01],
        [1.6839e-03],
        [3.8954e-02],
        [6.5933e-03],
        [3.2759e-04],
        [1.7551e-03],
        [3.2718e-04]], dtype=torch.float64)
Group distribution in result :  {'American': 196, 'French/Italian/European': 176, 'Fusion food(Mexican/mediterranean/caribbean)': 30, 'bbq/fast food': 18, 'Asian': 69, 'Diners/Cafes': 12}
Skew metrix :  1.24261059703135


1.24261059703135

In [23]:
#Tweak weights
weights2 = algorithm_gradient_descent(W,S,1e-5,100,-100,1,True,500)

Iteration 10 completed 
Iteration 20 completed 
Iteration 30 completed 
Iteration 40 completed 
Iteration 50 completed 
Iteration 60 completed 
Iteration 70 completed 
Iteration 80 completed 
Iteration 90 completed 
Iteration 100 completed 


In [24]:
print(weights2)
fair_calc(weights2,S,True,500)

tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], dtype=torch.float64)
Group distribution in result :  {'French/Italian/European': 177, 'bbq/fast food': 25, 'American': 169, 'Asian': 58, 'Fusion food(Mexican/mediterranean/caribbean)': 46, 'Diners/Cafes': 26}
Skew metrix :  0.49317710868141795


0.49317710868141795

In [ ]:
def comp_overlap(w1,w2,k):
  rls_matrix = get_RLS(w1,S)
  cand['score'] = rls_matrix.numpy()
  candy1 = cand.sort_values(by=['score'], ascending=False)

  # print(candy)

  dw1 = dict()
  temp1 = candy1.to_numpy()
  
  rls_matrix = get_RLS(w2,S)
  cand['score'] = rls_matrix.numpy()
  candy2 = cand.sort_values(by=['score'], ascending=False)

  # print(candy)
  dw2 = dict()
  temp2 = candy2.to_numpy()
  overlp = 0
  
  # print(candy1)
  # print(candy2)

  # print(temp1[0][1],temp1[0][2])
  # print(temp1)
  # print(temp2)
  # return 
  dname = {}

  
  for _i,i in enumerate(temp1):
    if(_i>k-1):
      break
    dname[str(i[1])+"#"+str(i[2])] = 1
  
  # print(len(dname))

  for _j,j in enumerate(temp2):
      if(_j>k-1):
        break
      if(str(j[1])+"#"+str(j[2]) in  dname.keys()):
        overlp+=1
  
  return overlp


In [ ]:
i=0
w_list = []

for k in [250,500,750,1000]:
  w_list.append(algorithm_gradient_descent(W,S,1e-5,100,-100,1,True,k))
  i+=1

Iteration 10 completed 
Iteration 20 completed 
Iteration 30 completed 
Iteration 40 completed 
Iteration 50 completed 
Iteration 60 completed 
Iteration 70 completed 
Iteration 80 completed 
Iteration 90 completed 
Iteration 100 completed 
Iteration 10 completed 
Iteration 20 completed 
Iteration 30 completed 
Iteration 40 completed 
Iteration 50 completed 
Iteration 60 completed 
Iteration 70 completed 
Iteration 80 completed 
Iteration 90 completed 
Iteration 100 completed 
Iteration 10 completed 
Iteration 20 completed 
Iteration 30 completed 
Iteration 40 completed 
Iteration 50 completed 
Iteration 60 completed 
Iteration 70 completed 
Iteration 80 completed 
Iteration 90 completed 
Iteration 100 completed 
Iteration 10 completed 
Iteration 20 completed 
Iteration 30 completed 
Iteration 40 completed 
Iteration 50 completed 
Iteration 60 completed 
Iteration 70 completed 
Iteration 80 completed 
Iteration 90 completed 
Iteration 100 completed 


In [ ]:
weight_file = open(method+'weights_list', 'wb') 
pickle.dump(w_list, weight_file) 
weight_file.close()

In [ ]:
weight_file = open(method+'weights1', 'wb') 
pickle.dump(weights1, weight_file) 
weight_file.close()

weight_file = open(method+'weights2', 'wb') 
pickle.dump(weights2, weight_file) 
weight_file.close()

In [ ]:
weight_file = open(method+'weights1', 'rb') 
weights1 = pickle.load(weight_file)

weight_file = open(method+'weights2', 'rb') 
weights2 = pickle.load(weight_file)

In [ ]:
l = get_RLS(weights2,S)
se = set()
print(l.shape)
for i in range(l.shape[0]):
    se.add(float(l[i,0]))
print(se)
print(len(se))

torch.Size([4951, 1])
{nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na

In [ ]:
path_G = path_variable+"matches_fodors_zagats.csv"
G = em.read_csv_metadata(path_G, 
                        #  key='_id',
                         ltable=A, rtable=B, 
                         fk_ltable='fodors_id', fk_rtable='zagats_id')

# print(G)
print(len(G))

gdic = {}
cnt_g = 0

G1 = G.to_numpy()
for i,x in enumerate(G1):
  s = str(x[0])+"#"+str(x[1])  
  s1 = str(x[1])+"#"+str(x[0])
  # print(s)
  gdic[s] = 1
  gdic[s1] = 1

print("Positive in ground truth",len(gdic)/2)
print(gdic.keys())

def check_gt(w1,S,k):
  rls_matrix = get_RLS(w1,S)
  cand['score'] = rls_matrix.numpy()
  candy1 = cand.sort_values(by=['score'], ascending=False)
  temp1 = candy1.to_numpy()
  
  overlp = 0
  neg = 0

  for _j,j in enumerate(temp1):
      if(_j>k-1):
        break
      s = str(j[1])+"#"+str(j[2])

      if(s in  gdic.keys()):
        # print(s)
        if(gdic[s]==1):
          overlp+=1
        else:
          neg+=1

  return overlp,neg

Metadata file is not present in the given path; proceeding to read the csv file.


112
Positive in ground truth 112.0
dict_keys(['534#219', '219#534', '535#220', '220#535', '536#221', '221#536', '537#222', '222#537', '538#223', '223#538', '539#224', '224#539', '540#225', '225#540', '541#226', '226#541', '542#227', '227#542', '543#228', '228#543', '544#229', '229#544', '545#230', '230#545', '546#231', '231#546', '547#232', '232#547', '548#233', '233#548', '549#234', '234#549', '550#235', '235#550', '551#236', '236#551', '552#237', '237#552', '553#238', '238#553', '554#239', '239#554', '555#240', '240#555', '556#241', '241#556', '557#242', '242#557', '558#243', '243#558', '559#244', '244#559', '560#245', '245#560', '561#246', '246#561', '562#247', '247#562', '563#248', '248#563', '564#249', '249#564', '565#250', '250#565', '566#251', '251#566', '567#252', '252#567', '568#253', '253#568', '569#254', '254#569', '570#255', '255#570', '571#256', '256#571', '572#257', '257#572', '573#258', '258#573', '574#259', '259#574', '575#260', '260#575', '576#261', '261#576', '577#262

In [ ]:
weight_file = open(method+'weights_list', 'rb') 
w_list = pickle.load(weight_file)

i=0
for k in [250,500,750,1000]:
  print("Top-",k)
  weight_this = w_list[i]
  i+=1
  # print("Weights =",weight_this)
  print("Fairness skew for w1 " + " is "+ str(fair_calc(weights1,S,False,k)))
  print("Fairness skew for w" + str(k)+ " is "+ str(fair_calc(weight_this,S,False,k)))
  print("Overlap with base result = "+str(comp_overlap(weights1,weight_this,k))+" /"+ str(k))
  print("Overlap with ground truth weights1", check_gt(weights1,S,k))
  print("Overlap with ground truth weights_this", check_gt(weight_this,S,k),"\n")


Top- 250
Fairness skew for w1  is 0.9507510704887621
Fairness skew for w250 is 0.7102760628229126
Overlap with base result = 8 /250
Overlap with ground truth weights1 (95, 0)
Overlap with ground truth weights_this (0, 0) 

Top- 500
Fairness skew for w1  is 1.24261059703135
Fairness skew for w500 is 0.49317710868141795
Overlap with base result = 44 /500
Overlap with ground truth weights1 (102, 0)
Overlap with ground truth weights_this (0, 0) 

Top- 750
Fairness skew for w1  is 1.3611492050697311
Fairness skew for w750 is 0.20657853443951454
Overlap with base result = 453 /750
Overlap with ground truth weights1 (103, 0)
Overlap with ground truth weights_this (104, 0) 

Top- 1000
Fairness skew for w1  is 1.063200250444854
Fairness skew for w1000 is 0.22129770840124618
Overlap with base result = 388 /1000
Overlap with ground truth weights1 (103, 0)
Overlap with ground truth weights_this (105, 0) 

